In [6]:
import nbimporter
import Face_Recogonization_Train as train
from keras.models import model_from_json
import cv2
from PIL import Image
import glob
import sys
import os
import sqlite3
import numpy as np
detector=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye=cv2.CascadeClassifier('haarcascade_eye.xml')
mouth=cv2.CascadeClassifier('haarcascade_mouth.xml')
car=cv2.CascadeClassifier('haarcascade_car.xml')

In [7]:
warning=input("Are you new User(y/n): ")
if warning=='y':
    model=train.Neural()
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")
    print("Saved model to disk")    

Are you new User(y/n): n


In [8]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [9]:
def imresize(path):
    a_i_p=glob.glob(os.path.join(path))
    i=0
    for i_p in a_i_p:
        img=Image.open(i_p)
        img=img.resize((100,100), Image.ANTIALIAS)
        img.save(i_p)
            
def take_data(path):
    imgs=[]
    all_img_path=glob.glob(os.path.join(path))
    
    np.random.shuffle(all_img_path)
    for img_path in all_img_path:
        img=cv2.imread(img_path,0)
        imgs.append(img)    
        
    X = np.array(imgs)
    newX=[]
    
    for k in range(0,len(X)):
        newX.append(X[k])
    newX=np.array(newX)
    
    return(newX)



In [ ]:
db=sqlite3.connect('database.db')
cursor = db.cursor()
cap = cv2.VideoCapture(0)
i=0
j=[]
face=0
facial = ""
conf = ""
while True:
    ret, frame = cap.read()
    gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces1 = detector.detectMultiScale(gray1, 1.3, 5)
    for (x,y,w,h) in faces1:
        
        if face !=0:
            cv2.putText(frame, facial, (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, conf, (x + 150, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray1=gray1[y:y+h, x:x+w]
        roi_color1=frame[y:y+h, x:x+w]
        cv2.imwrite('test/test.png', roi_color1) 
        
    cv2.imshow('frame',frame)
    imresize("test/test.png")
    test_X=take_data("test/test.png")
    test_X=np.reshape(test_X,[-1,100,100,1])
    test_X=test_X/255
    predict_classes=loaded_model.predict(test_X)
    if len(predict_classes)!=0:
        conf = str(int((np.max(predict_classes)/np.sum(predict_classes))*100))+"%"
        predict_classes=np.argmax(np.round(predict_classes),axis=1)
        sql='SELECT * FROM student WHERE Sno="'+str(int(predict_classes)+1)+'"'
        cursor.execute(sql)
        data=cursor.fetchall()
        facial=data[0][1]
        face=face+1     
        if int(predict_classes)+1 not in j:
            sys.stdout.write('\r')
            sys.stdout.write("ID: "+str(data[0][0])+"\nFirst Name: "+str(data[0][1])+"\nLast Name: "+str(data[0][2])+"\nEmail Id: "+str(data[0][3])+"\nBranch: "+str(data[0][4])+"\n\n")
            sys.stdout.flush()
            j.append(int(predict_classes)+1)
                   
        '''
        if labels[int(predict_classes)]=='Anshul':
            if i==0:
                engine = pyttsx3.init()

                engine.say("Hello Sir ")
                sql='SELECT * FROM EMPLOYEE WHERE first_name="'+str(labels[int(predict_classes)])+'"'
                cursor.execute(sql)
                data=cursor.fetchall()
                print ('First Name: '+str(data[0][0])+'\nLast Name: '+str(data[0][1])+'\nAge: '+str(data[0][2])+'\nSex: '+str(data[0][3])+'\nIncome: '+str(data[0][4]))
                engine.setProperty('rate',120)  #120 words per minute
                engine.setProperty('volume',0.9) 
                engine.runAndWait()
                i=i+1
                j=0
            sys.stdout.flush()
            facial=labels[int(predict_classes)]
    

        else:
            if j==0:
                engine = pyttsx3.init()

                engine.say("Sorry sir i can't recognize you ")

                engine.setProperty('rate',120)  #120 words per minute
                engine.setProperty('volume',1.5) 
                engine.runAndWait()
                i=0
                j=j+1
            facial='Unrecognize Face'
            '''
        os.remove('test/test.png')
    '''
    else:
        sys.stdout.write('\r')
        sys.stdout.write("Image Doesn't Capture due to face out of the frame or low light")
        sys.stdout.flush()
    '''   
    face=face+1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\nThank You and have a nice day :)")
        break
db.commit()
db.close()    
cap.release()
cv2.destroyAllWindows()

ID: 1
First Name: Anshul
Last Name: Gupta
Email Id: ganshul917
Branch: IT

ID: 2
First Name: Narender
Last Name: Modi
Email Id: nmodi@pmo.com
Branch: BJP

